# Chat

In [46]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_checkpoint = "NousResearch/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [36]:
# 引入必要的类型提示
from typing import List, Literal, TypedDict

# 从transformers库中导入PreTrainedTokenizer类
# PreTrainedTokenizer是一个通用的分词器类，用于处理文本和模型的输入
from transformers import PreTrainedTokenizer

# 定义角色的类型
# 角色可以是 "system"、"user" 或 "assistant"
Role = Literal["system", "user", "assistant"]

# 定义消息的结构
# 消息包括角色和内容
class Message(TypedDict):
    role: Role
    content: str

# 定义消息列表类型
# 一个消息列表是一个Message对象的列表
MessageList = List[Message]

# 定义用于标记不同部分的字符串
BEGIN_INST, END_INST = "[INST] ", " [/INST] "
BEGIN_SYS, END_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

def convert_list_of_message_lists_to_input_prompt(
    list_of_message_lists: List[MessageList], 
    tokenizer: PreTrainedTokenizer
) -> List[str]:
    """
    将多个消息列表转换为模型输入的提示字符串
    :param list_of_message_lists: 包含消息列表的列表，每个消息列表代表一个对话
    :param tokenizer: 用于处理文本的分词器
    :return: 转换后的输入提示字符串列表
    """
    input_prompts: List[str] = []  # 初始化用于存储输入提示的列表

    # 打印类型信息，用于调试
    print(type(list_of_message_lists))
    print(type(list_of_message_lists[0]))    

    # 遍历每个消息列表
    for message_list in list_of_message_lists:
        # 处理系统消息，将其与下一条消息合并
        if message_list[0]["role"] == "system":
            content = "".join([BEGIN_SYS, message_list[0]["content"], END_SYS, message_list[1]["content"]])
            message_list = [{"role": message_list[1]["role"], "content": content}] + message_list[2:]

        # 检查消息列表的角色顺序是否正确
        if not (
            all([msg["role"] == "user" for msg in message_list[::2]])  # 用户消息必须在偶数索引
            and all([msg["role"] == "assistant" for msg in message_list[1::2]])  # 助理消息必须在奇数索引
        ):
            raise ValueError(
                "Format must be in this order: 'system', 'user', 'assistant' roles.\nAfter that, you can alternate between user and assistant multiple times"
            )

        # 获取分词器的eos_token和bos_token
        eos = tokenizer.eos_token
        bos = tokenizer.bos_token

        # 构建输入提示字符串
        input_prompt = "".join(
            [
                "".join([bos, BEGIN_INST, (prompt["content"]).strip(), END_INST, (answer["content"]).strip(), eos])
                for prompt, answer in zip(message_list[::2], message_list[1::2])
            ]
        )

        # 确保最后一条消息来自用户
        if message_list[-1]["role"] != "user":
            raise ValueError(f"Last message must be from user role. Instead, you sent from {message_list[-1]['role']} role")

        # 添加最后一条用户消息到输入提示
        input_prompt += "".join([bos, BEGIN_INST, (message_list[-1]["content"]).strip(), END_INST])

        # 将构建好的输入提示添加到列表中
        input_prompts.append(input_prompt)

    return input_prompts


In [37]:
# 创建一个系统消息对象
# Message是一个TypedDict类型，定义了角色和内容
system_message = Message()
system_message["role"] = "system"  # 设置消息角色为 "system"
system_message["content"] = "Answer only with emojis"  # 设置消息内容
print(system_message)  # 打印系统消息

# 创建一个用户消息对象
user_message = Message()
user_message["role"] = "user"  # 设置消息角色为 "user"
user_message["content"] = "Who won the 2016 baseball World Series?"  # 设置消息内容
print(user_message)  # 打印用户消息

# 创建一个助理消息对象
# 这部分代码被注释掉了，如果需要使用，可以取消注释
# assistant_message = Message()
# assistant_message.role = "assistant"  # 设置消息角色为 "assistant"
# assistant_message.content = ""  # 设置消息内容（目前为空）

# 创建消息列表并添加消息
list_of_messages = list()  # 初始化消息列表
list_of_messages.append(system_message)  # 将系统消息添加到消息列表
list_of_messages.append(user_message)  # 将用户消息添加到消息列表

# 创建消息列表的列表
# 这是为了与convert_list_of_message_lists_to_input_prompt函数的参数类型匹配
list_of_message_lists = list()  # 初始化消息列表的列表
list_of_message_lists.append(list_of_messages)  # 将消息列表添加到消息列表的列表中

# 使用convert_list_of_message_lists_to_input_prompt函数生成输入提示
# 需要提供一个tokenizer对象，这里假设tokenizer已经定义
prompt = convert_list_of_message_lists_to_input_prompt(list_of_message_lists, tokenizer)
print(prompt)  # 打印生成的输入提示


{'role': 'system', 'content': 'Answer only with emojis'}
{'role': 'user', 'content': 'Who won the 2016 baseball World Series?'}
<class 'list'>
<class 'list'>
['<|begin_of_text|>[INST] <<SYS>>\nAnswer only with emojis\n<</SYS>>\n\nWho won the 2016 baseball World Series? [/INST] ']


In [49]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
# # 如果需要加载模型，可以使用以下代码
# model = AutoModelForCausalLM.from_pretrained("NousResearch/Meta-Llama-3.1-8B-Instruct")
# # 这行代码将加载与分词器配套的自回归语言模型

import torch
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)

# model = model.eval()

In [19]:
# 从transformers库中导入pipeline函数
# pipeline用于简化使用预训练模型的过程
from transformers import pipeline

# 使用tokenizer将输入提示转换为模型可以处理的格式
# tokenized_prompt是一个字典，其中包含了输入提示的token IDs等信息
tokenized_prompt = tokenizer(prompt)

# 打印输入提示的token数量
# "input_ids"是tokenizer输出的一部分，包含token IDs
# [0]表示我们取第一个示例（如果有多个示例的话）
# len(tokenized_prompt["input_ids"][0])计算token的数量
print(f'prompt is {len(tokenized_prompt["input_ids"][0])} tokens')


prompt is 31 tokens


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model=model_checkpoint)

In [ ]:
pipe(prompt)